In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy.stats as stats
import missingno as msno
import re
import math

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [2]:
tr = pd.read_csv("datasets/train.csv")
tr_test = pd.read_csv("datasets/test.csv")

tr['training'] = 1
tr_test['training'] = 0
tr_all = tr.append(tr_test)

***Data Cleaning***

In [3]:
def convert_to_snake_case(df):
    #add a space between any lowercase-capital letter pair, then replace spaces with _, the all to lowercase
    new_cols = {col: re.sub(r"([a-z]{1})([A-Z]{1})", r"\1 \2", col).replace(" ", "_").lower() for col in df.columns}
    return df.rename(columns = new_cols, inplace = True)

In [4]:
def zero_counts(df):
    counts = {col: df[col].value_counts().to_dict().get(0) for col in df.columns if df[col].value_counts().to_dict().get(0) != None}
    return {key: value for key, value in counts.items() if value > 0}

In [5]:
def neg_counts(df):
    counts = {col: sum([val for key, val in df[col].value_counts().items() if type(key) in [float, int] and key < 0]) for col in df.columns}
    return {key: value for key, value in counts.items() if value > 0}

In [6]:
def yes_no_to_bin(word):
    true_words = ['yes', 'true', 'y', 't']
    false_words = ['no', 'false', 'n', 'f']
    
    if str(word).lower() in true_words:
        return 1
    elif str(word).lower() in false_words:
        return 0
    else:
        return word

In [7]:
def get_corr_above_or_below(df, percentage):
    corr = df.corr()
    col_rows = corr.columns.tolist()
    
    keep_corrs = {}
    for col in col_rows:
        for row in col_rows:
            if corr.loc[col, row] >= percentage or corr.loc[col, row] <= percentage*-1:
                if col == row:
                    break
                if col in keep_corrs:
                    keep_corrs[col][row] = corr.loc[col, row]
                else:
                    keep_corrs[col] = {row: corr.loc[col, row]}
                    
    return keep_corrs

In [8]:
def outlier_dict(df, deviation = 3):
    cols = df.dtypes.to_dict()
    
    outlier_locations = {}
    for col, dtype in cols.items():
        if dtype in [np.float64, np.int64]:
            locations = np.abs(stats.zscore(df[col])) > deviation #credit to: https://stackoverflow.com/questions/23199796/detect-and-exclude-outliers-in-pandas-data-frame
            out_rows = df[locations]
            
            for row in out_rows.iterrows():
                loc = df.index.get_loc(row[0])
                if col in outlier_locations:
                    outlier_locations[col].append(loc)
                else:
                    outlier_locations[col] = [loc]
                    
    return outlier_locations

In [9]:
def outlier_dict_to_df(dictionary, original_df):
    out_cols = original_df.columns.tolist()
    out_cols.append("outlier_column_name")
    outliers = pd.DataFrame(columns = out_cols)

    for col, indices in dictionary.items():
        for index in indices:
            row = original_df.iloc[index].to_dict()
            row['outlier_column_name'] = col
            outliers = outliers.append(row, ignore_index = True)
            #outliers.style.applymap('background-color: yellow', outliers.iloc[outliers.shape[0] - 1, outliers.columns.get_loc(col)])outliers = outlier_dict_to_df(outlier_dict(sac), sac)

    return outliers

In [10]:
def corrs_selection(df, col, threshold = .5, greater_than = True):
    sale_corr = df.corr().to_dict()[col]
    if greater_than:
        sale_corr = {key: val for key, val in sale_corr.items() if val > threshold}
    else:
        sale_corr = {key: val for key, val in sale_corr.items() if val < threshold}
        
    return pd.DataFrame(sale_corr, index =[0])

In [11]:
convert_to_snake_case(tr_all)

In [12]:
tr_all = tr_all.applymap(yes_no_to_bin)

# NaN Cleaning

In [13]:
{key: val for key, val in tr_all[tr_all['training'] == 1].isna().sum().to_dict().items() if int(val) > 0}

{'lot_frontage': 330,
 'alley': 1911,
 'mas_vnr_type': 22,
 'mas_vnr_area': 22,
 'bsmt_qual': 55,
 'bsmt_cond': 55,
 'bsmt_exposure': 58,
 'bsmt_fin_type_1': 55,
 'bsmt_fin_sf_1': 1,
 'bsmt_fin_type_2': 56,
 'bsmt_fin_sf_2': 1,
 'bsmt_unf_sf': 1,
 'total_bsmt_sf': 1,
 'bsmt_full_bath': 2,
 'bsmt_half_bath': 2,
 'fireplace_qu': 1000,
 'garage_type': 113,
 'garage_yr_blt': 114,
 'garage_finish': 114,
 'garage_cars': 1,
 'garage_area': 1,
 'garage_qual': 114,
 'garage_cond': 114,
 'pool_qc': 2042,
 'fence': 1651,
 'misc_feature': 1986}

Rows to dummy

- Lot frontage has NaNs that cannot be 0, since all houses have to be connected to the street. It is highly correlated to lot area, which is the lot's square feet. We can make the assumption that all houses are roughly square for the purposes of approximating lot_frontage, as frontage represents one side and thus the root.

In [14]:
corrs_selection(tr_all[tr_all['training'] == 1], 'lot_frontage')

,lot_frontage,lot_area
0,1.0,0.574024


In [15]:
front = tr_all['lot_frontage'].tolist()
area = tr_all['lot_area'].to_list()

for i in range(0, len(front)):
    if math.isnan(front[i]):
        front[i] = area[i]**.5

tr_all['lot_frontage'] = pd.Series(front)

- Alley is nominal, and NaN values are meaningful in the data dictionary. We will dummy the column.
- Mas Vnr Type is nominal with meaningful NaNs. No removal necessary.
- Mas Vnr Area is always NaN when Mas Vnr Type is. Type being NaN indicates no veneer type, so we will set veneer area to 0.

In [16]:
tr_all['mas_vnr_area'] = tr_all['mas_vnr_area'].map(lambda x: x if not math.isnan(x) else 0)

- Basement sf1, sf2, unf sf, total sf, full bath, and half bath are all continuous data. The only nulls in those values are in rows where bsmt_exposure is NaN, indicating that there is no basement. These values will be set to 0.

In [34]:
tr_all[tr_all['bsmt_full_bath'].isna()][['training', 'bsmt_qual', 'bsmt_cond', 'bsmt_exposure', 'bsmt_fin_type_1', 'bsmt_fin_sf_1', 'bsmt_fin_type_2', 'bsmt_fin_sf_2', 'bsmt_unf_sf', 'total_bsmt_sf', 'bsmt_full_bath', 'bsmt_half_bath']]

,training,bsmt_qual,bsmt_cond,bsmt_exposure,bsmt_fin_type_1,bsmt_fin_sf_1,bsmt_fin_type_2,bsmt_fin_sf_2,bsmt_unf_sf,total_bsmt_sf,bsmt_full_bath,bsmt_half_bath
616,1,NaN,NaN,NaN,NaN,0.0,NaN,0.0,0.0,0.0,NaN,NaN
1327,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
basement_nums = ['bsmt_fin_sf_1','bsmt_fin_sf_2', 'bsmt_unf_sf', 'total_bsmt_sf', 'bsmt_full_bath', 'bsmt_half_bath']
for col in basement_nums:
    tr_all[col] = tr_all[col].map(lambda x: x if not math.isnan(x) else 0)

- Basement quality, condition, exposure, type1, and type2 are all ordinal values with meaningful NaNs. They will be dummied later.
- Fireplace qu is ordinal with a meaningful NaN.
- Garage cars and garage area are discrete values that are only NaN in a single row. That row has all it's garage ordinal and nominal values indicating that there is no garage except for garage type. We will set cars and area to 0 and type to NaN to be in line with other, similar rows.

In [43]:
tr_all[tr_all['garage_cars'].isna()][['training', 'garage_type', 'garage_yr_blt', 'garage_finish', 'garage_cars', 'garage_area', 'garage_qual', 'garage_cond']]

,training,garage_type,garage_yr_blt,garage_finish,garage_cars,garage_area,garage_qual,garage_cond
1712,1,Detchd,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
tr_all.iloc[1712, tr_all.columns.get_loc('garage_area')] = 0
tr_all.iloc[1712, tr_all.columns.get_loc('garage_cars')] = 0
tr_all.iloc[1712, tr_all.columns.get_loc('garage_type')] = np.NaN

- Garage type, year built, finish, quality, and condition all have meaningful NaNs that will be dummied later.
- Pool qc, fence, and misc feature also full under this category. There are no more NaNs for us to clean.

In [56]:
{key: val for key, val in tr_all[tr_all['training'] == 1].isna().sum().to_dict().items() if int(val) > 0}

{'alley': 1911,
 'mas_vnr_type': 22,
 'bsmt_qual': 55,
 'bsmt_cond': 55,
 'bsmt_exposure': 58,
 'bsmt_fin_type_1': 55,
 'bsmt_fin_type_2': 56,
 'fireplace_qu': 1000,
 'garage_type': 114,
 'garage_yr_blt': 114,
 'garage_finish': 114,
 'garage_qual': 114,
 'garage_cond': 114,
 'pool_qc': 2042,
 'fence': 1651,
 'misc_feature': 1986}

In [ ]:
objects = {}
for key, val in tr.dtypes.to_dict().items():
    if val == np.dtype('O'):
        objects[key] = tr[key].nunique()

In [ ]:
sum(objects.values())

In [ ]:
c = get_corr_above_or_below(tr, .5)
all_good_corrs = pd.DataFrame(c)
all_good_corrs

In [ ]:
sale_corrs = corrs_selection(tr, 'sale_price')
sale_corrs.drop(columns = ['sale_price'], inplace = True)
sale_corrs

In [ ]:
colinears = []
sale_cols = sale_corrs.columns.tolist()
all_colls = all_good_corrs.columns.tolist()
all_rows = list(all_good_corrs[all_colls[0]].to_dict().keys())

for col in sale_cols:
    if col in all_colls:
        for row in sale_cols:
            if row in all_rows and col != row and not math.isnan(all_good_corrs[col][row]):
                colinears.append((col, row))

In [ ]:
colinear_counts = {}
for a, b in colinears:
    colinear_counts[a] = colinear_counts[a] + 1 if a in colinear_counts else 1
    colinear_counts[b] = colinear_counts[b] + 1 if b in colinear_counts else 1
    
high_colinear_counts = []
low_colinear_counts = []

for key, val in colinear_counts.items():
    if val > 4:
        high_colinear_counts.append(key)
    else:
        low_colinear_counts.append(key)

In [ ]:
x_list = good_corrs.columns.tolist()
X = tr[x_list]
y = tr['sale_price']
lr = LinearRegression()
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [ ]:
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
r2_score(y_test, y_pred)

In [ ]:
x_list = high_colinear_counts
X = tr[x_list]
y = tr['sale_price']
lr = LinearRegression()
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
r2_score(y_test, y_pred)

In [ ]:
x_list = low_colinear_counts
X = tr[x_list]
y = tr['sale_price']
lr = LinearRegression()
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
r2_score(y_test, y_pred)

In [ ]:
high_colinear_counts